In [357]:
# importing the packages
import pandas as pd
from datetime import date
from functools import reduce

### Q1

In [358]:
#importing all 4 data sets
jan=pd.read_csv("C://Users//rxr318//Desktop//OYO//TableA.csv")
feb=pd.read_csv("C://Users//rxr318//Desktop//OYO//TableB.csv")
march=pd.read_csv("C://Users//rxr318//Desktop//OYO//TableC.csv")
tableD=pd.read_excel("C://Users//rxr318//Desktop//OYO//TableD.xlsx")

In [359]:
#Checking the data types of all columns in jdataset jan
jan.dtypes

booking_id       int64
customer_id      int64
source           int64
status           int64
checkin         object
checkout        object
oyo_rooms        int64
hotel_id         int64
amount           int64
discount       float64
date            object
room_nights    float64
dtype: object

In [360]:
# convert checkin,checkout and date columns to date time from object for all tables
jan[["checkin", "checkout","date"]] = jan[["checkin", "checkout","date"]].apply(pd.to_datetime)
feb[["checkin", "checkout","date"]] = feb[["checkin", "checkout","date"]].apply(pd.to_datetime)
march[["checkin", "checkout","date"]] = march[["checkin", "checkout","date"]].apply(pd.to_datetime)

In [361]:
#checking datatypes again
jan.dtypes

booking_id              int64
customer_id             int64
source                  int64
status                  int64
checkin        datetime64[ns]
checkout       datetime64[ns]
oyo_rooms               int64
hotel_id                int64
amount                  int64
discount              float64
date           datetime64[ns]
room_nights           float64
dtype: object

### Q2

In [261]:
#unique users for each month
unique_users=pd.DataFrame()
unique_users_jan=pd.DataFrame({'jan_unique':jan['customer_id'].unique()})
unique_users_feb=pd.DataFrame({'feb_unique':feb['customer_id'].unique()})
unique_users_march=pd.DataFrame({'march_unique':march['customer_id'].unique()})

#concatinating unique users for each month into the same dataframe
unique_users=pd.concat([unique_users_jan,unique_users_feb,unique_users_march],axis=1)

#exporting list of unique users to csv
unique_users.to_csv("C://Users//rxr318//Desktop//OYO//Q2_Unique_users_monthly.csv")

In [318]:
#finding number of bookings and amount spent for each customer for jan, feb and march
jan_agg=jan.groupby(['customer_id'],as_index=False).agg({'booking_id':{'booking_id.jan':'nunique'},'amount':{'amount.jan':'sum'}})
feb_agg=feb.groupby(['customer_id'],as_index=False).agg({'booking_id':{'booking_id.feb':'nunique'},'amount':{'amount.feb':'sum'}})
march_agg=march.groupby(['customer_id'],as_index=False).agg({'booking_id':{'booking_id.march':'nunique'},'amount':{'amount.march':'sum'}})


C:\Users\rxr318\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [245]:
#creating a function to calculate room nights for each booking
def _room_nights(dataset):
    #Converting checkout and checkin to datetime.date
    dataset['checkin']=dataset['checkin'].apply(lambda x:datetime.datetime.date(x))
    dataset['checkout']=dataset['checkout'].apply(lambda x:datetime.datetime.date(x))
    #Finding the room_nights and stoeing it in the respective dataset
    dataset['room_nights']=dataset['oyo_rooms']*(dataset['checkout']-dataset['checkin'])
    

In [246]:
#Finding roomnights for jan,feb,march
_room_nights(jan)
_room_nights(feb)
_room_nights(march)

In [362]:
#Checking the dataset for addition of a new column
jan.head()

,booking_id,customer_id,source,status,checkin,checkout,oyo_rooms,hotel_id,amount,discount,date,room_nights
0,1,189314,4,3,2017-02-04,2017-02-05,1,252,3160,669.0,2017-01-31,1.0
1,2,46268,4,3,2017-01-27,2017-01-28,1,252,1893,481.0,2017-01-27,1.0
2,3,55271,3,2,2017-01-25,2017-01-26,1,252,2188,463.0,2017-01-25,1.0
3,4,170766,4,3,2017-01-26,2017-01-27,1,252,3054,646.0,2017-01-26,1.0
4,5,170766,4,3,2017-01-26,2017-01-28,1,252,6107,1293.0,2017-01-25,2.0


In [321]:
#calculating total room nights stayed(status 2) for all users in all months and stoeing it in month_room_niht_stayed
jan_room_night_stayed=jan[jan['status']==2].groupby(['customer_id'],as_index=False).agg({'room_nights':{'room_night.jan':'sum'}})
feb_room_night_stayed=feb[feb['status']==2].groupby(['customer_id'],as_index=False).agg({'room_nights':{'room_night.feb':'sum'}})
march_room_night_stayed=feb[feb['status']==2].groupby(['customer_id'],as_index=False).agg({'room_nights':{'room_night.feb':'sum'}})

C:\Users\rxr318\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


### Q3

In [335]:
#storing all individual datframes into a variable
dfs = [jan_agg,jan_room_night_stayed,feb_agg,feb_room_night_stayed,march_agg,march_room_night_stayed]

In [350]:
#merging all data sets together at customer level
df_final = reduce(lambda left,right: pd.merge(left,right,on='customer_id',how='outer'), dfs)

C:\Users\rxr318\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:3108: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [351]:
#dropping the first level of column names
df_final.columns=df_final.columns.droplevel(0)

In [354]:
#Renaming the first column of the final dataset
df_final.rename(columns={ df_final.columns[0]: "Customer_id" })
#Exporting the final datset to csv
df_final.to_csv("C://Users//rxr318//Desktop//OYO//Q3_Final_Dataset.csv")

### Q4

In [161]:
#repeat rate

#storing number of unique customers in january in variable X
X=jan['customer_id'].nunique()

#Storing number customers who shopped in Jan and then in feb in Y
Y=feb.loc[feb['customer_id'].isin(jan['customer_id'])]['customer_id'].nunique()

In [164]:
#finding repeat rate and storin it in X
repeat_rate=Y/X*100

In [165]:
#disply the repeat rate
repeat_rate

18.497913769123784

### Q5


In [170]:
#taking the union of jan,feb and march data to find revenue for the entire time period
unionABC=pd.concat([jan,feb,march],ignore_index=True)

C:\Users\rxr318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [174]:
#joining the union table with city mapping
city_booking=pd.merge(left=unionABC,right=tableD,on='hotel_id',how='inner')

In [198]:
#finding the sum of amount, grouping it at city and hotel level, the aggregated column is renamed as amount_sum
city_agg=city_booking.groupby(['city','hotel_id']).agg({'amount':{'amount_sum':'sum'}})
#dropping the 1st level of column names after aggregation
city_agg.columns = city_agg.columns.droplevel(0)

C:\Users\rxr318\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [215]:
#arranging the amount_sum column in descending order and keeping only top 3 rows for each city
top_3_citywise=city_agg.sort_values(['city','amount_sum'],ascending=False).groupby('city').head(3)

In [216]:
#exporting the final dataset for Q5 to csv
city_agg.to_csv("C://Users//rxr318//Desktop//OYO//Q5_top_3_citywise.csv")